<a href="https://colab.research.google.com/github/simandi2877/eva/blob/master/Project9/Assignment9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U -q PyDrive ## you will have install for every colab session
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
from keras import backend as K
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, SeparableConv2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils

import tensorflow as tf
from tensorflow.python.framework import ops


from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#https://drive.google.com/open?id=1MdOVBUQoTiEl1WcOE9v7moWTcbEDeLEB
  
hdf5file = drive.CreateFile({'id':'1MdOVBUQoTiEl1WcOE9v7moWTcbEDeLEB'})
hdf5file.GetContentFile('weights.hdf5')
#!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
#!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#!apt-get update -qq 2>&1 > /dev/null
#!apt-get -y install -qq google-drive-ocamlfuse fuse
#from google.colab import auth
#auth.authenticate_user()
#from oauth2client.client import GoogleCredentials
#creds = GoogleCredentials.get_application_default()
#import getpass
#!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
#vcode = getpass.getpass()
#!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
#from google.colab import drive

# This will prompt for authorization.
#drive.mount('/content/drive')
#!ls "/content/drive/My Drive"




W0626 18:35:35.062670 139711014328192 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lock

In [3]:
print( os.getcwd() )


import h5py
#filepath = h5py.File(hdf5file, 'r')

def build_model():
  model = Sequential()
  #When 3x3 convolutions are used, Receptive field = (2**(i+1) − 1)×(2**(i+1) − 1) where i is the dilation rate.

  model.add(Convolution2D(32, 3, 3, input_shape=(32, 32, 3))) #30
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))

  model.add(Convolution2D(64, 3, 3, activation='relu')) #28
  model.add(BatchNormalization())
  model.add(Dropout(0.2))

  model.add(Convolution2D(64, 3, 3, activation='relu')) #26
  model.add(BatchNormalization())
  model.add(Dropout(0.2))

  model.add(Convolution2D(64, 3, 3, activation='relu')) #24
  model.add(BatchNormalization())
  model.add(Dropout(0.2))

  model.add(Convolution2D(64, 3, 3, activation='relu')) #22
  model.add(BatchNormalization())
  model.add(Dropout(0.2))

  model.add(MaxPooling2D(pool_size=(2, 2))) #11
  model.add(Dropout(0.2))

  model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu')) #9
  model.add(BatchNormalization())
  model.add(Dropout(0.2))

  #model.add(SeparableConv2D(64, 3, 3, activation='relu', border_mode='same' )) #9
  #model.add(Dropout(0.2))

  model.add(MaxPooling2D(pool_size=(2, 2))) #5
  model.add(Dropout(0.25))

  model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu')) #3
  model.add(BatchNormalization())
  model.add(Dropout(0.2))


  #model.add(MaxPooling2D(pool_size=(2, 2)))
  #model.add(Dropout(0.25))


  #model.add(Convolution2D(128, 1,1, activation='relu', border_mode='same'))
  #model.add(BatchNormalization())
  #model.add(Dropout(0.1))

  model.add(Convolution2D(10, 5))
  model.add(Flatten())
  model.add(Activation('softmax'))

  from keras.callbacks import LearningRateScheduler
  import math
  def scheduler(epoch, lr):
    return round(0.003 * 1/(1 + 0.319 * epoch), 10)
  # Compile the model
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  
  model.load_weights('weights.hdf5')

  return model

H, W = 224, 224 # Input shape, defined by the model (model.input_shape)

/content


In [4]:
model = build_model()
model.summary()

W0626 18:35:50.150400 139711014328192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3...)`
  # This is added back by InteractiveShellApp.init_path()
W0626 18:35:50.177438 139711014328192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0626 18:35:50.181237 139711014328192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0626 18:35:50.216746 139711014328192 deprecation_wrap

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
dropout_2 (Dropout)          (None, 28, 28, 64)        0         
__________